# Модель GARCH(1,1) Skewed-t

- Обучение на returns
- Прогноз волатильности (скользящее окно)
- Tail risk

Шаблон содержит структуру для быстрого запуска, визуализирует сходится ли модель.


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Попытка импорта arch для GARCH
try:
    from arch import arch_model
    print("✅ Библиотеки загружены (arch доступен)")
    ARCH_AVAILABLE = True
except ImportError:
    print("⚠️ Библиотека 'arch' не установлена. Установите: pip install arch")
    ARCH_AVAILABLE = False


✅ Библиотеки загружены (arch доступен)


## Загрузка данных


In [ ]:
import json

DATA_DIR = Path('../02_feature_engineering/data/features')

# Вариант 1: Получить список тикеров из файлов в директории
all_files = list(DATA_DIR.glob("*_with_targets.parquet"))
tickers = [f.stem.replace("_with_targets", "") for f in all_files]

# Вариант 2: Или загрузить из metadata (более надёжный способ)
# with open(Path('../config/tickers_metadata.json'), 'r') as f:
#     metadata = json.load(f)
# tickers = [k for k in metadata.keys() if not k.startswith('_')]

# Загрузка всех данных в словарь
all_data = {}
for ticker in tickers:
    file_path = DATA_DIR / f"{ticker}_with_targets.parquet"
    if file_path.exists():
        df = pd.read_parquet(file_path)
        # Подготовка данных для GARCH
        returns = df['log_return'].dropna() * 100  # Масштабируем для стабильности
        all_data[ticker] = {
            'df': df,
            'returns': returns
        }
        print(f"✅ {ticker}: {len(returns)} наблюдений, mean={returns.mean():.4f}, std={returns.std():.4f}")

print(f"\n📊 Загружено {len(all_data)} тикеров: {list(all_data.keys())}")



📊 Загружено 0 тикеров: []


## Обучение GARCH(1,1) модели


In [ ]:
if ARCH_AVAILABLE:
    garch_models = {}
    
    for ticker, data in all_data.items():
        returns = data['returns']
        
        # Создаем и обучаем GARCH(1,1) модель с Skewed-t распределением
        model = arch_model(returns, vol='Garch', p=1, q=1, dist='skewt', rescale=False)
        
        try:
            fitted_model = model.fit(disp='off', show_warning=False)
            
            # Извлечение условной волатильности
            conditional_vol = fitted_model.conditional_volatility
            
            # Прогноз волатильности на 1 шаг вперед
            forecast = fitted_model.forecast(horizon=1)
            forecast_variance = forecast.variance.iloc[-1, 0]
            forecast_vol = np.sqrt(forecast_variance)
            
            garch_models[ticker] = {
                'model': fitted_model,
                'conditional_vol': conditional_vol,
                'forecast_vol': forecast_vol
            }
            print(f"✅ {ticker}: прогноз волатильности = {forecast_vol:.4f}%")
            
        except Exception as e:
            print(f"❌ {ticker}: ошибка обучения - {e}")
    
    print(f"\n📊 Успешно обучено моделей: {len(garch_models)}/{len(all_data)}")
    
else:
    print("❌ GARCH модель недоступна. Установите библиотеку 'arch'.")

## Walk-forward прогнозирование


In [ ]:
if ARCH_AVAILABLE:
    # Walk-forward прогнозирование на последних 100 днях
    train_size = len(returns) - 100
    forecasts = []
    
    print("Walk-forward прогнозирование (может занять время)...")
    
    for i in range(train_size, len(returns)):
        train_data = returns.iloc[:i]
        
        # Обучаем модель на историческом окне
        model_temp = arch_model(train_data, vol='Garch', p=1, q=1, dist='skewt', rescale=False)
        res_temp = model_temp.fit(disp='off', show_warning=False, update_freq=0)
        
        # Прогноз на 1 шаг
        forecast_temp = res_temp.forecast(horizon=1)
        forecasts.append(np.sqrt(forecast_temp.variance.iloc[-1, 0]))
    
    # Добавляем прогнозы в DataFrame
    forecast_dates = df['date'].iloc[-100:]
    forecast_df = pd.DataFrame({
        'date': forecast_dates.values,
        'garch_forecast': forecasts
    })
    
    print(f"✅ Walk-forward прогнозирование завершено: {len(forecasts)} прогнозов")
    print(forecast_df.tail())
    
else:
    print("❌ Walk-forward недоступен без библиотеки 'arch'.")


## Сохранение результатов


In [ ]:
if ARCH_AVAILABLE:
    OUTPUT_DIR = Path('data') / 'models'
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    # Сохранение прогнозов
    output_path = OUTPUT_DIR / f"{ticker}_garch_forecasts.parquet"
    forecast_df.to_parquet(output_path, index=False)
    
    print(f"✅ Сохранено: {output_path}")
else:
    print("⚠️ Нечего сохранять без GARCH модели")
